## Deliverable 2. Create a Customer Travel Destinations Map

In [1]:
# Dependencies and Setup
import pathlib
import sys

import gmaps
import pandas as pd
import requests

# Import API key
sys.path.append('..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

### Import City and Weather Data

In [2]:
# 1. Import the WeatherPy_database.csv file. 
# keep_default_na is false to avoid changing the country code for Namibia from
# NA to NaN. The na_values list is used to provide some common NaN type values
# in case they exist in other columns of the csv.
city_data_df = pd.read_csv(
    pathlib.Path("../Weather_Database/WeatherPy_Database.csv"),
    na_values=['NULL', 'NaN', 'n/a', 'nan', 'null'],
    keep_default_na=False
)
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Busselton,AU,-33.6500,115.3333,66.04,81,25,12.46,scattered clouds
1,Catumbela,AO,-12.4314,13.5472,79.93,66,84,8.63,broken clouds
2,Rikitea,PF,-23.1203,-134.9692,77.36,75,100,14.70,overcast clouds
3,Guerrero Negro,MX,27.9769,-114.0611,63.86,73,100,6.24,overcast clouds
4,Dunmore East,IE,52.1514,-6.9861,49.68,77,75,6.91,broken clouds


### Retrieve User Preferences

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  75
What is the maximum temperature you would like for your trip?  90


### Apply User Preferences

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a 
# new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp)
                                       & (city_data_df['Max Temp'] <= max_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty
# rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City                   211
Country                211
Lat                    211
Lng                    211
Max Temp               211
Humidity               211
Cloudiness             211
Wind Speed             211
Current Description    211
dtype: int64

### Add Empty Hotel Column

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city,
# country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", 
                                "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Catumbela,AO,79.93,broken clouds,-12.4314,13.5472,
2,Rikitea,PF,77.36,overcast clouds,-23.1203,-134.9692,
8,San Pedro,PH,76.32,scattered clouds,14.3500,121.0167,
10,Arraial Do Cabo,BR,78.66,moderate rain,-22.9661,-42.0278,
14,Singkang,MY,76.87,overcast clouds,2.1006,102.7994,
18,Thinadhoo,MV,83.39,overcast clouds,0.5333,72.9333,
19,Mar Del Plata,AR,75.00,clear sky,-38.0023,-57.5575,
23,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,
26,Townsville,AU,81.48,clear sky,-19.2500,146.8000,
27,Buala,SB,80.28,light rain,-8.1450,159.5921,


### Retrieve Nearest Hotels

In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:
    # 6e. Make request and retrieve the JSON data from the search.
        hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel
    # isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
-----------------------------
Data Retrieval Complete      
-----------------------------


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.drop(index=hotel_df[hotel_df['Hotel Name'] == ''].index)

### Export Vacation Data

In [10]:
# 8a. Create the output File (CSV)
output_data_file = pathlib.Path("WeatherPy_Vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Create Marker Layer with Info Box

In [11]:
# 9. Using the template add city name, the country code, the weather description
# and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and
# store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new
# DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))